***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-99ab  GCE       4                                             RUNNING  us-central1-a


To take a quick anonymous survey, run:
  $ gcloud survey



# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar 10 14:05 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'jhonny_328783105'
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
# base_dir = SparkFiles.getRootDirectory()
blobs = client.list_blobs(bucket_name)
# for b in blobs:
#     if b.name != 'graphframes.sh' :
#         paths.append(full_path+b.name)
for b in blobs:
    if b.name.endswith('.parquet'):
        paths.append(full_path + b.name)


In [8]:
base_dir = 'body/posting_locations_folder'


***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [9]:
parquetFile = spark.read.parquet(*paths)


In [10]:
from pyspark import StorageLevel

doc_body_pairs = parquetFile.select("text", "id").rdd


We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [10]:
# Count number of wiki pages
# parquetFile.count()
# sc.stop()

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [11]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [12]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [13]:
from inverted_index_gcp import InvertedIndex

In [ ]:
from pyspark import SparkContext
import re
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# No need to download stopwords every time if already available
# nltk.download('stopwords')
sc = SparkContext.getOrCreate()

# Regular expression for words
RE_WORD = re.compile(r"""[\w#@]+(['\-]?[\w#@]+)*""", re.UNICODE)

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
broadcasted_stopwords = sc.broadcast(all_stopwords)

def process_partition(iterator):
    stemmer = PorterStemmer()
    for body, doc_id in iterator:
        tokens = [token.group() for token in RE_WORD.finditer(body.lower())]
        stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in broadcasted_stopwords.value]
        yield (doc_id, len(stemmed_tokens))

def calculate_doc_lengths(doc_body_pairs):
    # Now doc_body_pairs is expected to have (body, doc_id) tuples
    doc_lengths_rdd = doc_body_pairs.mapPartitions(process_partition)
    
    doc_lengths_rdd.cache()  # Cache since we use it multiple times
    
    total_documents, total_length = doc_lengths_rdd.aggregate(
        (0, 0),  # Initial values for the count and sum
        lambda acc, val: (acc[0] + 1, acc[1] + val[1]),  # seqOp
        lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])  # combOp
    )
    
    # Calculate the average document length
    average_doc_length = total_length / total_documents if total_documents else 0

    return total_documents, average_doc_length, doc_lengths_rdd

# Execute the function with your actual RDD variable name for doc_body_pairs
total_documents, average_doc_length, doc_lengths_rdd = calculate_doc_lengths(doc_body_pairs)


In [ ]:
total_documents

In [ ]:
# doc_lengths_rdd.saveAsTextFile(f'gs://{bucket_name}/partitionned_doc_lenghts_body')


In [ ]:
# from google.cloud import storage
# import pickle

# def aggregate_and_pickle(bucket_name, input_path, output_path):
#     client = storage.Client()
#     bucket = client.bucket(bucket_name)
#     blobs = bucket.list_blobs(prefix=input_path)

#     aggregated_dict = {}
#     for blob in blobs:
#         for line in blob.download_as_text().splitlines():
#             doc_id, body_len = eval(line)  # Use eval carefully; ensure your data is safe
#             aggregated_dict[doc_id] = body_len

#     # Pickle the aggregated dictionary
#     pickle_data = pickle.dumps(aggregated_dict)
#     blob = bucket.blob(output_path)
#     blob.upload_from_string(pickle_data)

# # Example usage
# aggregate_and_pickle(bucket_name, 'partitionned_doc_lenghts_body', 'dict_lenghts_body_.pickle')


In [2]:
# doc_lengths_rdd

NameError: name 'doc_lengths_rdd' is not defined

In [ ]:
# print(total_documents)

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [ ]:
#Word count
import nltk
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

the_inverted_index = InvertedIndex()

english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
NUM_BUCKETS = 124

def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

def word_count(text, id, stemmer):
    """
    Count the frequency of each word in `text` after stemming and excluding stopwords,
    and return entries that will go into our posting lists.

    Parameters:
    - text: str - Text of one document
    - id: int - Document id
    - stemmer: PorterStemmer instance for stemming words

    Returns:
    List of tuples - A list of (stemmed_token, (doc_id, tf)) pairs
    """
    # Tokenize the text using a regular expression
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    
    # Apply stemming and filter out stopwords
    stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in all_stopwords]

    # Count the frequency of each stemmed word
    word_freq = Counter(stemmed_tokens)

    # Format the result as a list of (stemmed_word, (doc_id, frequency)) pairs
    result = [(word, (id, freq)) for word, freq in word_freq.items()]
    
    return result

from pyspark.sql import SparkSession
from pyspark import SparkContext
from math import log

from pyspark.sql import SparkSession
from math import log
def calculate_bm25_components(term_freqs_rdd, df_dict, total_docs,avgdl, doc_length_rdd, k1=1.5, b=0.75, k3=1.5):
    """
    Calculate BM25 component for each term-document pair without using collectAsMap() for doc_length_rdd.
    
    Parameters:
    - term_freqs_rdd: RDD of the form [('term1', ('doc_id1', frequency)), ('term1', ('doc_id2', frequency)), ...]
    - df_dict_broadcast: Broadcasted dictionary of document frequencies
    - total_docs: Total number of documents in the corpus
    - doc_length_rdd: RDD of document lengths [('doc_id1', length), ('doc_id2', length), ...]
    - k1, b: BM25 parameters
    """
    # Calculate average document length
#     avgdl = doc_length_rdd.map(lambda x: x[1]).mean()
    df_dict_broadcast = sc.broadcast(df_dict)

    # Convert term frequencies RDD to include df and total_docs using the broadcasted df_dict
    def include_df(term_doc_freq):
        term, (doc_id, freq) = term_doc_freq
        df = df_dict_broadcast.value.get(term, 0)
        return (doc_id, (term, freq, df, total_docs))

    # Map term_freqs_rdd to include df and total_docs, then join with doc_length_rdd
    tf_with_df_rdd = term_freqs_rdd.map(include_df)
    joined_rdd = tf_with_df_rdd.join(doc_length_rdd)

    # Calculate BM25 component using the joined RDD
    def calculate_bm25(data):
        doc_id, ((term, freq, df, total_docs), doc_length) = data
        idf = log((total_docs - df + 0.5) / (df + 0.5) + 1)
        denom = (1 - b + b * doc_length / avgdl) * k1 + freq
        bm25 = idf * ((k1 + 1)*(k3 + 1) * freq) / denom
        return (term, (doc_id, bm25))

    bm25_components_rdd = joined_rdd.map(calculate_bm25).groupByKey().mapValues(list)
    
    return bm25_components_rdd

# Note: This is a conceptual example. Depending on your Spark version, you might need to adjust the API calls.
# Example usage stays similar to the previous function.

# def calculate_bm25_components(term_freqs_rdd, df_rdd, total_docs, avgl,doc_length_rdd, k1=1.5, b=0.75):
#     """
#     Calculate BM25 component for each term-document pair without using collectAsMap().

#     Parameters are the same as in the original function description.
#     """
    
#     # Calculate average document length
# #     avgdl = doc_length_rdd.map(lambda x: x[1]).mean()

#     # Convert DF rdd to a dictionary format in an RDD [(term, df), ...] -> [(term, (df, N)), ...]
#     df_with_total_docs_rdd = df_rdd.map(lambda x: (x[0], (x[1], total_docs)))
    
#     # Join term_freqs_rdd with df_rdd to combine term frequency and document frequency
#     term_df_freq_rdd = term_freqs_rdd.map(lambda x: (x[0], (x[1][0], x[1][1]))).join(df_with_total_docs_rdd)

#     # Now join with doc_length_rdd [(doc_id, length)] to add document lengths to the calculations
#     # First, adjust term_df_freq_rdd to have doc_id as the key
#     term_df_freq_rdd = term_df_freq_rdd.map(lambda x: (x[1][0][0], (x[0], x[1][0][1], x[1][1][0], x[1][1][1])))
    
#     # Join with doc_length_rdd
#     final_rdd = term_df_freq_rdd.join(doc_length_rdd)

#     # Calculate BM25 component
#     def bm25_term_doc(data):
#         doc_id, ((term, freq, df, total_docs), doc_length) = data
#         idf = log((total_docs - df + 0.5) / (df + 0.5) + 1)
#         B = 1 - b + b * (doc_length / avgdl)
#         bm25_component = ((k1 + 1) * freq) / (B * k1 + freq) * idf
#         return (term, (doc_id, bm25_component))

#     bm25_components_rdd = final_rdd.map(bm25_term_doc)
#     return bm25_components_rdd

# Note: This implementation assumes that you have the SparkContext (sc) available. Adjustments may be needed based on your actual SparkSession setup.

def reduce_word_counts(unsorted_pl):
    ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
    return sorted(unsorted_pl)



#calculate_df

def calculate_df(postings):
    ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
    df_rdd = postings.map(lambda x: (x[0], len(set(x[1]))))

    return df_rdd
def calculate_idf(postings, total_docs):
    import math
    ''' Takes a posting list RDD and calculate the idf for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
    idf_rdd = postings.map(lambda x: (x[0], math.log((total_docs- len(set(x[1])) + 0.5) / (len(set(x[1])) + 0.5) + 1)))
    return idf_rdd
def partition_postings_and_write(postings):
    ''' A function that partitions the posting lists into buckets, writes out
  all posting lists in a bucket to disk, and returns the posting locations for
  each bucket. Partitioning should be done through the use of `token2bucket`
  above. Writing to disk should use the function  `write_a_posting_list`, a
  static method implemented in inverted_index_colab.py under the InvertedIndex
  class.
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and
      offsets its posting list was written to. See `write_a_posting_list` for
      more details.
  '''

    postings_with_bucket_id = postings.map(lambda x: (token2bucket_id(x[0]), x))
    postings_grouped_by_bucket = postings_with_bucket_id.groupByKey().mapValues(list)

    def write_postings(bucket_id, postings_list):
        b_w_pl = (bucket_id, postings_list)
        return the_inverted_index.write_a_posting_list(b_w_pl,base_dir, bucket_name)
    
    posting_locations_rdd = postings_grouped_by_bucket.map(lambda x: write_postings(x[0], x[1]))

    return posting_locations_rdd
# import nltk
# from nltk.stem import PorterStemmer



In [ ]:

# time the index creation time
t_start = time()
# word counts map
# word_counts = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts = doc_body_pairs.flatMap(lambda x: word_count(x[0], x[1], stemmer))

postings = word_counts.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
postings_filtered = postings.filter(lambda x: len(x[1])>50)
#df
w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()
# idf

bm25_components_rdd = calculate_bm25_components(word_counts, w2df_dict, total_documents,average_doc_length, doc_lengths_rdd)
# with_bm25_postings = bm25_components_rdd.groupByKey().mapValues(reduce_word_counts)
# filtering postings and calculate df
with_bm25_postings_filtered = bm25_components_rdd.filter(lambda x: len(x[1])>50)
# partition posting lists and write out
index_const_time = time() - t_start

In [ ]:
_ = partition_postings_and_write(with_bm25_postings_filtered).collect()


In [20]:
doc_body_pairs.take(10)

[Row(text='\'\'\'Foster Air Force Base\'\'\' (1941–1945, 1952–1959) is a former United States Air Force facility in Texas, located in Victoria County, approximately  east-northeast of Victoria.\n\nA\xa0flying training airfield during World War II, it was part of Tactical Air Command (TAC) during the early years of the Cold War as a tactical fighter and command base.\n\nThe airfield honored Lt. Arthur L. Foster (25 November 1888 - 10 February 1925), a Texas native from Georgetown. A\xa0U.S. Army Air Corps instructor, he was killed in a crash at Brooks Field, near San Antonio. Foster\'s son received his training and commission at the namesake base in\xa01942.\n\nWorld War II\nthumb|Formation of 12 AT-6 trainers over the Guadalupe River near Foster\xa0Field, Summer\xa01942\nThe airfield was established in the spring of 1941 as an advanced single-engine flying school for fighter pilots. A\xa0local funding campaign led by E. J. Dysart the previous spring had raised some $17,000 to locate th

In [17]:
bm25_components_rdd.take(100)

[('divid',
  [(7370312, 1, 23.187945134077914),
   (22510960, 1, 14.097875004929971),
   (55968888, 1, 23.187945134077914),
   (3104216, 1, 19.969023429231665),
   (1662344, 1, 27.644048213989574),
   (40207993, 1, 19.969023429231665),
   (44809882, 1, 27.644048213989574),
   (215019, 1, 23.187945134077914),
   (16297827, 1, 27.644048213989574),
   (1476851, 1, 19.969023429231665),
   (32416086, 1, 23.187945134077914),
   (23691199, 1, 19.969023429231665),
   (3213853, 1, 17.534856463001912),
   (8210823, 1, 14.097875004929971),
   (43362591, 1, 19.969023429231665),
   (20485583, 1, 15.629647088516279),
   (35341042, 1, 19.969023429231665),
   (31323938, 1, 23.187945134077914),
   (27462826, 1, 19.969023429231665),
   (4646344, 1, 27.644048213989574),
   (36038184, 1, 27.644048213989574),
   (54382000, 1, 23.187945134077914),
   (14344144, 1, 23.187945134077914),
   (11240176, 1, 27.644048213989574),
   (55830287, 1, 14.097875004929971),
   (1743494, 1, 23.187945134077914),
   (3829919

In [22]:
# test index construction time
# assert index_const_time < 60*120
index_const_time

7465.342294692993

SyntaxError: invalid syntax (1990861999.py, line 1)

In [21]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='body/posting_locations_folder'):
#     print('yes')
    if not blob.name.endswith("pickle"):
#         print('no')
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)

In [23]:
super_posting_locs

defaultdict(list,
            {'ritzer': [('body/posting_locations_folder/0_000.bin', 0)],
             'llanov': [('body/posting_locations_folder/0_000.bin', 592)],
             'szilard': [('body/posting_locations_folder/0_000.bin', 1616)],
             'zubak': [('body/posting_locations_folder/0_000.bin', 3144)],
             'galai': [('body/posting_locations_folder/0_000.bin', 3632)],
             'eun-bi': [('body/posting_locations_folder/0_000.bin', 4416)],
             'argentino': [('body/posting_locations_folder/0_000.bin', 5728)],
             'chocolat': [('body/posting_locations_folder/0_000.bin', 36760)],
             '0043': [('body/posting_locations_folder/0_000.bin', 43352)],
             'weichselian': [('body/posting_locations_folder/0_000.bin',
               44896)],
             'louco': [('body/posting_locations_folder/0_000.bin', 46184)],
             'kwamina': [('body/posting_locations_folder/0_000.bin', 47704)],
             'lubycza': [('body/posting_locatio

Putting it all together

In [24]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# idf
# inverted._base_dir = base_dir
# x = MultiFileReader()
# inverted.idf = w2idf_dict
# Add the doc_id - doc_length to the inverted index
# inverted.doc_lengths = doc_lengths
# Add N
# inverted.N = total_documents
# Add avgl
# inverted.avgl = average_doc_length
# write the global stats out
inverted.write_index('.', 'index')
# print(inverted.doc_lengths)
#updating the total num of docs and the avg 
# inverted.finalize_index()
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/body_index/{index_src}'
!gsutil cp $index_src $index_dst

Copying file://index.pkl [Content-Type=application/octet-stream]...
/ [1 files][ 16.0 MiB/ 16.0 MiB]                                                
Operation completed over 1 objects/16.0 MiB.                                     


In [25]:
!gsutil ls -lh $index_dst

 16.01 MiB  2024-04-08T10:20:44Z  gs://jhonny_328783105/body_index/index.pkl
TOTAL: 1 objects, 16788008 bytes (16.01 MiB)


In [ ]:
# inverted.read_index('', 'body_index/index',bucket_name)
inverted.posting_locs

In [27]:
# super_posting_locs
# inverted.posting_locs
inverted.read_a_posting_list('', 'receiv', bucket_name)
#     def read_a_posting_list(self, base_dir, w, bucket_name=None):


[(19480152, 6.526364803314209),
 (19698888, 3.1391289234161377),
 (20100152, 4.9522600173950195),
 (20104120, 3.2946090698242188),
 (19130720, 5.168786525726318),
 (41554384, 5.000813961029053),
 (63376400, 4.9522600173950195),
 (2332440, 2.2605576515197754),
 (12394296, 4.06899356842041),
 (30256000, 7.150146961212158),
 (30656272, 4.553422927856445),
 (47955512, 4.6017632484436035),
 (676048, 4.527003288269043),
 (10976728, 7.4863128662109375),
 (14827920, 3.8943333625793457),
 (28990952, 6.224389553070068),
 (44848816, 6.175126552581787),
 (45521888, 8.075167655944824),
 (61449440, 10.10102653503418),
 (60359480, 4.46483039855957),
 (21219128, 8.047707557678223),
 (21873104, 4.47507381439209),
 (22022648, 4.758943557739258),
 (22039016, 3.8564083576202393),
 (37153872, 4.865785121917725),
 (38366096, 6.717405796051025),
 (49698456, 6.763993263244629),
 (158224, 4.334594249725342),
 (191208, 5.788790702819824),
 (56961384, 7.227662086486816),
 (5579008, 4.908641815185547),
 (6814048,

# PageRank

**YOUR TASK (10 POINTS):** Compute PageRank for the entire English Wikipedia. Use your implementation for `generate_graph` function from Colab below.

In [22]:
# Put your `generate_graph` function here
def generate_graph(pages):
  ''' Compute the directed graph generated by wiki links.
  Parameters:
  -----------
    pages: RDD
      An RDD where each row consists of one wikipedia articles with 'id' and
      'anchor_text'.
  Returns:
  --------
    edges: RDD
      An RDD where each row represents an edge in the directed graph created by
      the wikipedia links. The first entry should the source page id and the
      second entry is the destination page id. No duplicates should be present.
    vertices: RDD
      An RDD where each row represents a vetrix (node) in the directed graph
      created by the wikipedia links. No duplicates should be present.
  '''

  # YOUR CODE HERE
  # raise NotImplementedError()
  edges = pages.flatMap(lambda page: [(page['id'], anchor_row['id']) for anchor_row in page['anchor_text']]).distinct()

  # Extract vertices from both the source page ids and destination page ids
  # Extract source page IDs
  source_vertices = pages.map(lambda page: (page['id'],)).distinct()
  # Extract destination page IDs
  destination_vertices = pages.flatMap(lambda page: [(anchor_row['id'],) for anchor_row in page['anchor_text']]).distinct()
  # Union source and destination vertices to get all unique vertices, then remove duplicates
  vertices = source_vertices.union(destination_vertices).distinct()

  return edges, vertices

In [ ]:
t_start = time()
pages_links = parquetFile.limit(1000).select ("id","anchor_text").rdd
# construct the graph 
edges, vertices = generate_graph(pages_links)
# compute PageRank
edgesDF = edges.toDF(['src', 'dst']).repartition(124, 'src')
verticesDF = vertices.toDF(['id']).repartition(124, 'id')
g = GraphFrame(verticesDF, edgesDF)
pr_results = g.pageRank(resetProbability=0.15, maxIter=6)
pr = pr_results.vertices.select("id", "pagerank")
pr = pr.sort(col('pagerank').desc())
pr.repartition(1).write.csv(f'gs://{bucket_name}/pr', compression="gzip")
pr_time = time() - t_start
pr.show()

In [ ]:
# test that PageRank computaion took less than 1 hour
assert pr_time < 120*60

# Reporting

**YOUR TASK (5 points):** execute and complete the following lines to complete 
the reporting requirements for assignment #3. 

In [ ]:
# size of input data
!gsutil du -sh "gs://wikidata_preprocessed/"

In [ ]:
# size of index data
index_dst = f'gs://{bucket_name}/postings_gcp/'
!gsutil du -sh "$index_dst"

In [ ]:
# How many USD credits did you use in GCP during the course of this assignment?
cost = 0.07
print(f'I used {cost} USD credit during the course of this assignment')

**Bonus (10 points)** if you implement PageRank in pure PySpark, i.e. without using the GraphFrames package, AND manage to complete 10 iterations of your algorithm on the entire English Wikipedia in less than an hour. 


In [ ]:
#If you have decided to do the bonus task - please copy the code here 

bonus_flag = False # Turn flag on (True) if you have implemented this part

t_start = time()

# PLACE YOUR CODE HERE

pr_time_Bonus = time() - t_start


In [ ]:
# Note:test that PageRank computaion took less than 1 hour
assert pr_time_Bonus < 60*60 and bonus_flag